In [1]:
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import requests
from bs4 import BeautifulSoup
import csv

In [5]:
def google_search_scraper(query, num_results=10):
    search_url = f"https://www.google.com/search?q={query}&num={num_results}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}
    
    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        print("Failed to fetch results")
        return

    soup = BeautifulSoup(response.text, "html.parser")
    results = []

    for g in soup.select('.tF2Cxc'):
        title = g.select_one('h3').text if g.select_one('h3') else None
        link = g.select_one('a')['href'] if g.select_one('a') else None
        description = g.select_one('.VwiC3b').text if g.select_one('.VwiC3b') else None

        if title and link:
            results.append({"Title": title, "Link": link, "Description": description})

    with open("google_search_results.csv", "w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["Title", "Link", "Description"])
        writer.writeheader()
        writer.writerows(results)

    print(f"Saved {len(results)} results to 'google_search_results.csv'")

In [6]:
google_search_scraper("Python web scraping", num_results=10)

Saved 8 results to 'google_search_results.csv'


In [7]:
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [21]:
def google_search_scraper_selenium(query, num_results=10, output_file="google_search_results.csv"):
    """Scrapes Google search results for the given query and saves them to a CSV file."""
    
    WAIT_TIME = 2
    
    options = Options()
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")  
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(service=service, options=options)
    print("Starting the browser and navigating to Google...")

    try:
        driver.get(f"https://www.google.com/search?q={query}&num={num_results}")
        time.sleep(WAIT_TIME)
        print("Fetching search results...")

        # Scrape search results
        results = []
        search_results = driver.find_elements(By.CSS_SELECTOR, ".tF2Cxc")

        for result in search_results:
            try:
                title = result.find_element(By.CSS_SELECTOR, "h3").text

                link = result.find_element(By.CSS_SELECTOR, "a").get_dom_attribute("href")
                
                description_element = result.find_elements(By.CSS_SELECTOR, ".VwiC3b")
                description = description_element[0].text if description_element else "No description available"

                if title and link:
                    results.append({"Title": title, "Link": link, "Description": description})
            except Exception as e:
                print(f"Error extracting a result: {e}")
                continue

        with open(output_file, "w", newline="", encoding="utf-8") as file:
            writer = csv.DictWriter(file, fieldnames=["Title", "Link", "Description"])
            writer.writeheader()
            writer.writerows(results)

        print(f"Saved {len(results)} results to '{output_file}'")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.quit()
        print("Browser closed.")

In [22]:
# Example usage
if __name__ == "__main__":
    google_search_scraper_selenium("Python web scraping tutorial", num_results=10)

Starting the browser and navigating to Google...
Fetching search results...
Saved 8 results to 'google_search_results.csv'
Browser closed.


In [ ]:
import pandas as pd
df = pd.DataFrame(pd.read_csv("google_search_results.csv"))
df

,Title,Link,Description
0,Python Web Scraping Tutorial,https://www.geeksforgeeks.org/python-web-scrap...,"11 Nov 2024 — In this tutorial, we'll explore ..."
1,Python Web Scraping Tutorial: Step-By-Step [20...,https://oxylabs.io/blog/python-web-scraping,5 Mar 2024 — For this Python web scraping tuto...
2,A Practical Introduction to Web Scraping in Py...,https://realpython.com/python-web-scraping-pra...,"In this tutorial, you'll learn all about web s..."
3,Python Web Scraping: Full Tutorial With Exampl...,https://www.scrapingbee.com/blog/web-scraping-...,28 May 2024 — Learn about web scraping in Pyth...
4,Python Web Scraping Tutorial,https://www.tutorialspoint.com/python_web_scra...,This tutorial will teach you various concepts ...
5,Python Web Scraping Tutorial – How to Scrape D...,https://www.freecodecamp.org/news/how-to-scrap...,10 Aug 2021 — You will learn how to inspect a ...
6,Web Scraping Using Python,https://www.javatpoint.com/web-scraping-using-...,BeautifulSoup is a Python library that is used...
7,Web Scraping With Python – 2024 Full Guide,https://brightdata.com/blog/how-tos/web-scrapi...,Building a Web Scraper in Python · Step 1: Cho...
